In [0]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import pandas as pd
# fix random seed for reproducibility
numpy.random.seed(7)

Using TensorFlow backend.


In [0]:
!pip install tensorflow-gpu==1.15

     |████████████████████████████████| 411.5MB 37kB/s 
     |████████████████████████████████| 3.8MB 51.2MB/s 
     |████████████████████████████████| 512kB 54.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=7f690f2f8d6afecae99781c29b467c72e69a694b358361ed9ee788c7ea3da485
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.

In [0]:
from keras.layers import CuDNNLSTM

In [0]:
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [0]:
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [0]:
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,22,21,134,476,26,480,5,144,30,2,18,51,36,28,224,92,25,104,4,226,65,16,38,1334,88,12,16,283,5,16,4472,113,103,32,15,16,2,19,178,32
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,152,491,18,2,32,2,1212,14,9,6,371,78,22,625,64,1382,9,8,168,145,23,4,1690,15,16,4,1355,5,28,6,52,154,462,33,89,78,285,16,145,95
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,23,22,12,272,40,57,31,11,4,22,47,6,2307,51,9,170,23,595,116,595,1352,13,191,79,638,89,2,14,9,8,106,607,624,35,534,6,227,7,129,113
3,687,23,4,2,2,6,3693,42,38,39,121,59,456,10,10,7,265,12,575,111,153,159,59,16,1447,21,25,586,482,39,4,96,59,716,12,4,172,65,9,579,...,14,31,9,242,955,48,25,279,2,23,12,1685,195,25,238,60,796,2,4,671,7,2804,5,4,559,154,888,7,726,50,26,49,2,15,566,30,579,21,64,2574
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,74,233,334,207,126,224,12,562,298,2167,1272,7,2601,5,516,988,43,8,79,120,15,595,13,784,25,3171,18,165,170,143,19,14,5,2,6,226,251,7,61,113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,112,84,7,1399,11,68,1427,11,2,2,2086,32,4,428,3834,11,4,20,485,51,739,8,30,2012,4045,456,1183,64,18,231,2,198,646,196,17,25,2537,12,17,1219
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,74,2,6,1573,1167,1347,15,2,96,8,76,23,2331,46,7,129,1140,2,35,4752,425,18,14,20,9,4,420,7,4,1012,2,13,482,15,2,2568,4,359,18,2
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,234,2766,234,1119,1574,7,496,4,139,929,2901,2,2,5,4241,18,4,2,2,250,11,1818,2,4,4217,2,747,1115,372,1890,1006,541,2,7,4,59,2,4,3586,2
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,14,123,5,942,25,8,721,12,145,5,202,12,160,580,202,12,6,52,58,2,92,401,728,12,39,14,251,8,15,251,5,2,12,38,84,80,124,12,9,23


In [0]:
pd.DataFrame(X_test)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,13,92,124,89,488,2,100,28,1668,14,31,23,27,2,29,220,468,8,124,14,286,170,8,157,46,5,27,239,16,179,2,38,32,25,2,451,202,14,6,717
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,529,48,25,181,8,67,35,1732,22,49,238,60,135,1162,14,9,290,4,58,10,10,472,45,55,878,8,169,11,374,2,25,203,28,8,818,12,125,4,3077
2,33,6,58,54,1270,431,748,7,32,2580,16,11,94,2,10,10,4,993,2,7,4,1766,2634,2164,2,8,847,8,1450,121,31,7,27,86,2663,2,16,6,465,993,...,2321,42,1898,11,4,3814,42,101,704,7,101,999,15,1625,94,2926,180,5,9,2,34,2,45,6,1429,22,60,6,1220,31,11,94,2,96,21,94,749,9,57,975
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7,149,102,15,26,814,38,465,1627,31,70,983,67,51,9,112,814,17,35,311,75,26,2,574,19,4,1729,23,4,268,38,95,138,4,609,191,75,28,314,1772
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,401,14,1361,879,13,28,8,401,61,1642,2925,44,1373,21,591,353,14,500,4092,30,290,12,10,10,65,790,790,206,158,300,45,15,52,2,158,692,2,158,856,158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,20,2,17,73,2035,49,7,4,1596,3856,7,119,71,1322,5,306,8,30,23,4,1289,7,2111,21,34,4,130,316,144,923,14,9,6,194,749,418,7,498,12,9,...,2,119,26,254,8,79,83,60,11,2,63,22,119,47,210,77,262,11,52,154,363,4,22,203,809,346,7,51,84,535,21,6,693,46,7,158,20,290,319,3563
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,14,1494,161,4363,33,32,279,4,1126,1014,7,32,58,562,273,17,4,2,2,2,27,182,425,429,4,2095,3745,50,26,120,1691,4259,11,32,38,25,79,3400,7,206
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,116,218,78,21,407,100,30,128,262,15,7,185,2280,284,1842,2,37,315,4,226,20,272,2942,40,29,152,60,181,8,30,50,553,362,80,119,12,21,846,2
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,985,200,5,2,5,9,1861,8,79,357,4,20,47,220,57,206,139,11,12,5,55,117,212,13,1276,92,124,51,45,1188,71,536,13,520,14,20,6,2302,7,470


In [0]:
from keras.optimizers import SGD,Adam

In [0]:
# create the model
opt= SGD(lr=0.4,momentum=0.0)
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(CuDNNLSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.summary())
h1=model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
cu_dnnlstm_3 (CuDNNLSTM)     (None, 100)               53600     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 213,701
Trainable params: 213,701
Non-trainable params: 0
_________________________________________________________________
None
Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 17s 689us/step - loss: 0.6945 - accuracy: 0.5012 - val_loss: 0.6934 - val_accuracy: 0.5000
Epoch 2/3
25000/25000 [==============================] - 17s 681us/step - loss: 0.6893 - accuracy: 0.5314 - val_loss: 0.6908 - val_accuracy: 0.5179
Epoch 3/3
25000/25000 [====

In [0]:
# create the model
opt= SGD(lr=0.4,momentum=0.5)
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(CuDNNLSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.summary())
h1=model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 100)               53600     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,701
Trainable params: 213,701
Non-trainable params: 0
_________________________________________________________________
None

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 19s 741us/step - loss: 0.6941 - accuracy: 0.5084 - val_loss: 0.6894 - val_accuracy: 0.527

In [0]:
# create the model
opt= SGD(lr=0.01,momentum=0.2)
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.summary())
h1=model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 286s 11ms/step - loss: 0.5577 - accuracy: 0.7048 - val_loss: 0.4444 - val_accuracy: 0.7944
Epoch 2/3
25000/25000 [==============================] - 286s 11ms/step - loss: 0.4218 - accuracy: 0.8100 - val_loss: 0.4301 - val_accuracy: 0.8052
Epoch 3/3
25000/25000 [==============================] - 283s 11ms/step - loss: 0.3716 - accuracy: 0.8370 - val_loss: 0.4708 - val_accuracy: 0.7817


In [0]:
# create the model
opt= SGD(lr=0.01,momentum=0.03)
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.summary())
h1=model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 287s 11ms/step - loss: 0.6941 - accuracy: 0.5528 - val_loss: 0.6968 - val_accuracy: 0.5473
Epoch 2/3
25000/25000 [==============================] - 288s 12ms/step - loss: 0.6673 - accuracy: 0.5929 - val_loss: 0.6651 - val_accuracy: 0.5992
Epoch 3/3
25000/25000 [==============================] - 288s 12ms/step - loss: 0.6326 - accuracy: 0.6345 - val_loss: 0.6778 - val_accuracy: 0.6043


In [0]:
# create the model
opt= SGD(lr=0.01,momentum=0.4)
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.summary())
h1=model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 282s 11ms/step - loss: nan - accuracy: 0.5845 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/3
25000/25000 [==============================] - 282s 11ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/3
25000/25000 [==============================] - 287s 11ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00


In [0]:
# create the model
opt= Adam(lr=0.06)
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.summary())
h6=model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 291s 12ms/step - loss: 0.6021 - accuracy: 0.6602 - val_loss: 0.5157 - val_accuracy: 0.7478
Epoch 2/3
25000/25000 [==============================] - 300s 12ms/step - loss: 0.5622 - accuracy: 0.7157 - val_loss: 0.5930 - val_accuracy: 0.6919
Epoch 3/3
25000/25000 [==============================] - 309s 12ms/step - loss: 0.5341 - accuracy: 0.7351 - val_loss: 0.5573 - val_accuracy: 0.7250


In [0]:
# create the model
opt= Adam(lr=0.07)
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 296s 12ms/step - loss: 0.5947 - accuracy: 0.6770 - val_loss: 0.7919 - val_accuracy: 0.5549
Epoch 2/3
25000/25000 [==============================] - 293s 12ms/step - loss: 0.6699 - accuracy: 0.6165 - val_loss: 0.6458 - val_accuracy: 0.6199
Epoch 3/3
25000/25000 [==============================] - 292s 12ms/step - loss: 0.6191 - accuracy: 0.6570 - val_loss: 0.6414 - val_accuracy: 0.6386


In [0]:
# create the model
opt= Adam(lr=0.08)
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
lstm_8 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 288s 12ms/step - loss: nan - accuracy: 0.0331 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/3
 9088/25000 [=========>....................] - ETA: 2:27 - loss: nan - accuracy: 0.0000e+00

KeyboardInterrupt: ignored